In [81]:
import sympy
from sympy import CC, QQ
import numpy as np
from sortedcollections import ValueSortedDict
from fractions import Fraction

In [82]:
# Ideal class example
f = sympy.sympify('z**3 + x**3 * y**3 + y**2 * z**2 + x**4')
f = sympy.sympify("x**2 + x*y*z")
g = sympy.Poly(f, domain = 'CC')
ring = sorted(list(f.free_symbols), key=lambda x: str(x))
n = len(ring) # number of vars, like x0, ..., xn
ring = CC.old_poly_ring(*ring)
ideal = ring.ideal(f)
print(ideal)

def ideal_to_monomial_terms(ideal:sympy.polys.agca.ideals.Ideal) -> np.ndarray:
    """Returns a flat list of powers corresponding to points in the newton polygon"""
    monomial_terms = np.array([term[0] for gen in ideal.gens for term in gen.terms()])
    return monomial_terms

def inv_dict(n_vars) -> ValueSortedDict:
    """Makes a dictionary such that dict[i][i] = np.inf else 0"""
    out = ValueSortedDict(sum)
    for i in range(n_vars):
        endpoint = [0]*n_vars
        endpoint[i] = np.inf
        out[i] = tuple(endpoint)
    return out

def project_used(point, outpoints:ValueSortedDict):
    """Given the current best restrictions project a point to a new possible
      best restriction. Pushes the points along restrictions"""
    for index, w_var in outpoints.items():
        if len(np.nonzero(point)[0]) == 1: #only one variable left
            return point
        if point[index] == 0 or w_var[index] >= np.inf:
            continue #nothing to project
        if abs(w_var[index] - point[index])<0.1e-10 or w_var[index] < point[index]:
            scale = np.inf #projection cannot hit axis
        else:
            scale = w_var[index] / (w_var[index] - point[index])
        point[index] = 0 
        with np.errstate(invalid='ignore'):
            point = point * scale
        point[np.isnan(point)] = 0 #np.inf*0 = 0
    return point


def calculate_inv_plane(points, verbose = False):
    n = points.shape[1]
    outpoints = inv_dict(n)
    for point in points:
        if verbose > 2:print("Point", point, "outpoints", outpoints)
        point = project_used(point, outpoints)  #project if possible
        index = np.nonzero(point)[0]
        degree = np.sum(point)
        for i in index: #handles both multiple or single variables
            if degree >= outpoints[i][i]:
                continue #only want more restrictive points
            newpoint = [0]*n
            newpoint[i] = degree
            outpoints[i] = tuple(newpoint)
    out = [outpoints[i][i] for i in range(n)]
    return out

def deg_sort(points):
    index = np.lexsort(
        (np.sum(points!=0, axis=1),
         np.sum(points, axis=1)))
    points = points[index]
    return points

def inv_finder(ideal:sympy.polys.agca.ideals.Ideal, verbose = False):
    vars = ideal.ring.symbols
    n = len(vars)
    points = ideal_to_monomial_terms(ideal) # returns (flat) list like [(a,b,c),...]
    points = deg_sort(points)
    if verbose >= 2:print(points)
    plane = calculate_inv_plane(points, verbose)
    if verbose >= 2:print(plane)
    inv = [Fraction(i).limit_denominator(1000) 
            if i < np.inf else np.inf for i in plane] 
    return inv

inv_finder(ideal, verbose=False)

<1.0*x**2 + 1.0*x*y*z>


[Fraction(2, 1), Fraction(4, 1), Fraction(4, 1)]

In [83]:
def make_ideal(*inputs, gen_out = False):
    vars = set()
    generators = []
    for s in inputs:
        f = sympy.sympify(s)
        vars = vars.union(f.free_symbols)
        generators.append(f)
    vars = sorted(list(vars), key=lambda x: str(x))
    ring = QQ.old_poly_ring(*vars)
    ideal = ring.ideal(*generators)
    if gen_out:
        return ideal, generators
    return ideal

In [84]:
make_ideal("x", "y")

<x,y>

In [85]:
def test_admisibility(ideal:sympy.polys.agca.ideals.Ideal, inv):
    gens = list(ideal.gens)
    for poly in gens:
        for term in poly.terms():
            if sum(term[0][i]/inv[i] for i in range(len(inv))) < 1-0.1e-10:
                print(f"AssertionError {sympy.pprint(poly), term} did not work with {inv}.")
                return False
    return True

In [86]:
def make_weights(inv):
    denoms_lcm = sympy.lcm([frac.denominator for frac in inv if frac != np.inf])
    temp_inv = [(frac*denoms_lcm) if frac != np.inf else np.inf for frac in inv]
    nume_lcm = sympy.lcm([frac.numerator for frac in temp_inv if frac != np.inf])
    out = [1/(frac/nume_lcm) if frac != np.inf else 0 for frac in temp_inv]
    return tuple(out)

In [87]:
make_weights((2,4,8))

(4, 2, 1)

In [88]:
inv_finder(make_ideal(*"x*y*z**2,x**2".split(",")))

[Fraction(2, 1), Fraction(6, 1), Fraction(6, 1)]

In [89]:
def blow_up(ideal, generators, weights, chart_index = 0, verbose = False, keep_vars = False, new_var = "u", fast = False):
    vars = ideal.ring.symbols
    n = len(vars)
    # Charts by default x0 chart with u corresponding to x0
    if keep_vars:
        #new_vars = vars[:chart_index] + (sympy.sympify(new_var),) + vars[chart_index+1:]
        new_vars = vars[:]
    else:
        new_vars = sympy.symbols(' '.join([f"y{i}" if i != chart_index else new_var for i in range(n)]))
    u = new_vars[chart_index]
    u_var = {vars[chart_index]:u**weights[chart_index]}
    loc_vars = {vars[i]:u**weight * new_vars[i] if i != chart_index 
                else u for i, weight in enumerate(weights)}
    if verbose >= 1:
        print("====", "chart ", vars[chart_index], "====")
        print(loc_vars)

    
    new_variety = [gen.subs(u_var).subs(loc_vars) for gen in generators]
    all_powers = [term.as_coeff_exponent(u)[1] if term.has(u) 
                  else 0 
                  for eq in new_variety
                  for term in eq.as_ordered_terms()] # Finds the number of u's one can pull out

    # Calculate the minimum of the powers
    max_power = min(all_powers)

    # Divide each term by u raised to the max_power
    simplified_expressions = [poly / (u**max_power) 
                           for poly in new_variety]
    if verbose >= 1:print(simplified_expressions)
    if fast:return simplified_expressions
    # Simplify the divided expression
    simplified_expressions = [sympy.simplify(divided_expression) 
                              for divided_expression in simplified_expressions]
    results = [(u**max_power) * simplified_expression 
               for simplified_expression in simplified_expressions]
    # Multiply the simplified expression by u raised to the max_power
    if verbose >= 1:
        print(results)
    return results, simplified_expressions
    

In [90]:
def multiblowup(ideal:str, ring = None, charts = [], verbose = False, secure = True, path = None, stop_depth = 0):
    """
    Input: monomial ideal, like "x**3, x*y**3,..., x2**2"
    output: tree as list, like [[x**1, x**3], [y**1, x**1]]
    Constructs a tree such that each leaf at the end is the path that led to it,
    by finding the invariant for each layer, and then for each chart repeat recursively.
    The amount of args is bad practice, but it most are for extra functionality
    That did not seem to warrent an extra function.
    Also, since we terminate iff 1\in I, it often terminates one stop "to late".
    Yet to be implemented is a function to calculate smoothness of I"""
    if ring == None:
        ideal, generators = make_ideal(*ideal.split(","), gen_out=True)
        ring = ideal.ring
    else:
        generators = ideal
        ideal = ring.ideal(*ideal)
    if ideal.contains(1): # should check if smooth also. Also will work for non-monomials if fixed
        if verbose > 0.5:print("with", ring.symbols, "though ", charts, "0 no longer singular")
        return charts
    if generators[0] == 1:
        return charts + ["success"]
    inv = inv_finder(ideal, verbose = verbose)
    if all(i == 1 for i in inv):
        if verbose > 0.5:print("with", ring.symbols, "though ", charts, "0 no longer singular")
        return charts
    if verbose > 0.5:print("invariant =", inv, f" at chart {charts}"*(len(charts)>1))
    if secure: 
        if not test_admisibility(ideal, inv):
            print("with", ring.symbols, "though ", charts, "ERROR")
            if secure>1:
                assert test_admisibility(ideal, inv)
    weights = make_weights(inv)
    if verbose > 0.5:print("Weights^{-1}", weights, ring.symbols)
    tree = []
    for index, weight in enumerate(weights):
        if path and len(charts) < len(path):
            path_index = path[len(charts)]
            if index != path_index:
                continue
            if weight == 0:
                return
        if stop_depth and len(charts) >= stop_depth:
            return charts + ["end"]
        if weight == 0:
            continue
        out = blow_up(ideal, generators, weights, chart_index=index, verbose = verbose, keep_vars=True, fast=True)
        chart_var = f"{ideal.ring.symbols[index]}**{weight}"
        leaf = multiblowup(out, ring, charts = charts + [chart_var], verbose = verbose, secure = secure, path=path, stop_depth=stop_depth)
        tree.append(leaf)
    return tree

In [91]:
multiblowup("b22*b33, b33*t, b22**2 * t, b23*t, b12*b22*t, b22* t**2, t**3")

[[[['b22**1', 'b12**1', 'b33**1'], ['b22**1', 'b12**1', 't**1']],
  [['b22**1', 'b22**1', 'b33**1'], ['b22**1', 'b22**1', 't**1']],
  [['b22**1', 'b23**1', 'b33**1'], ['b22**1', 'b23**1', 't**1']],
  ['b22**1', 'b33**2'],
  [['b22**1', 't**1', 'b12**1'],
   ['b22**1', 't**1', 'b22**1'],
   ['b22**1', 't**1', 'b23**1'],
   ['b22**1', 't**1', 'b33**1']]],
 [[['b23**1', 'b22**1', 'b33**1'], ['b23**1', 'b22**1', 't**1']],
  [['b23**1', 'b33**1', 'b22**1'], ['b23**1', 'b33**1', 't**1']],
  ['b23**1', 't**2']],
 [['b33**1', 'b22**1'], ['b33**1', 't**1']],
 [[['t**1', 'b12**1', 'b22**1'],
   ['t**1', 'b12**1', 'b23**1'],
   ['t**1', 'b12**1', 'b33**1'],
   ['t**1', 'b12**1', 't**1']],
  [['t**1', 'b22**1', 'b12**1'],
   ['t**1', 'b22**1', 'b23**1'],
   ['t**1', 'b22**1', 'b33**1'],
   ['t**1', 'b22**1', 't**1']],
  ['t**1', 'b23**2'],
  ['t**1', 'b33**2'],
  ['t**1', 't**2']]]

In [103]:
multiblowup("z*y**3, z*x**3, x**3", verbose=1)

invariant = [Fraction(3, 1), Fraction(4, 1), Fraction(4, 1)] 
Weights^{-1} (4, 3, 3) (x, y, z)
==== chart  x ====
{x: x, y: x**3*y, z: x**3*z}
[y**3*z, x**3*z, 1]
with (x, y, z) though  ['x**4'] 0 no longer singular
==== chart  y ====
{x: x*y**4, y: y, z: y**3*z}
[z, x**3*y**3*z, x**3]
invariant = [Fraction(3, 1), inf, Fraction(1, 1)] 
Weights^{-1} (1, 0, 3) (x, y, z)
==== chart  x ====
{x: x, y: y, z: x**3*z}
[z, x**3*y**3*z, 1]
with (x, y, z) though  ['y**3', 'x**1'] 0 no longer singular
==== chart  z ====
{x: x*z, y: y, z: z}
[1, x**3*y**3*z**3, x**3]
with (x, y, z) though  ['y**3', 'z**3'] 0 no longer singular
==== chart  z ====
{x: x*z**4, y: y*z**3, z: z}
[y**3, x**3*z**3, x**3]
invariant = [Fraction(3, 1), Fraction(3, 1), inf] 
Weights^{-1} (1, 1, 0) (x, y, z)
==== chart  x ====
{x: x, y: x*y, z: z}
[y**3, z**3, 1]
with (x, y, z) though  ['z**3', 'x**1'] 0 no longer singular
==== chart  y ====
{x: x*y, y: y, z: z}
[1, x**3*z**3, x**3]
with (x, y, z) though  ['z**3', 'y**1'] 0 no

[['x**4'],
 [['y**3', 'x**1'], ['y**3', 'z**3']],
 [['z**3', 'x**1'], ['z**3', 'y**1']]]

In [93]:
### EXAMPLE ### 
ideal, generators = make_ideal("z*y**3","z*x**3","x**3", gen_out=True)
inv = inv_finder(ideal)
print("invariant =", inv)
test_admisibility(ideal, inv)
weights = make_weights(inv)
print("Weights^{-1}", weights)
blow_up(ideal, generators, weights, chart_index=1, verbose = True)
generators, res = blow_up(ideal, generators, weights, chart_index=0, verbose = True)
res[-1] == 1

invariant = [Fraction(3, 1), Fraction(4, 1), Fraction(4, 1)]
Weights^{-1} (4, 3, 3)
==== chart  y ====
{x: u**4*y0, y: u, z: u**3*y2}
[y2, u**3*y0**3*y2, y0**3]
[u**12*y2, u**15*y0**3*y2, u**12*y0**3]
==== chart  x ====
{x: u, y: u**3*y1, z: u**3*y2}
[y1**3*y2, u**3*y2, 1]
[u**12*y1**3*y2, u**15*y2, u**12]


True

In [94]:
### EXAMPLE ### 
ideal, generators = make_ideal("x**5 + x**3 * y**3 + y**8", gen_out=True)
inv = inv_finder(ideal)
print("invariant =", inv)
test_admisibility(ideal, inv)
weights = make_weights(inv)
print("Weights^{-1}", weights)
blow_up(ideal, generators, weights, chart_index=1, verbose = True)
blow_up(ideal, generators, weights, chart_index=0, verbose = True)
ideal

invariant = [Fraction(5, 1), Fraction(15, 2)]
Weights^{-1} (3, 2)
==== chart  y ====
{x: u**3*y0, y: u}
[(u**16 + u**15*y0**5 + u**15*y0**3)/u**15]
[u**15*(u + y0**5 + y0**3)]
==== chart  x ====
{x: u, y: u**2*y1}
[(u**16*y1**8 + u**15*y1**3 + u**15)/u**15]
[u**15*(u*y1**8 + y1**3 + 1)]


<x**5 + x**3*y**3 + y**8>

In [97]:
### EXAMPLE ### 
ideal, generators = make_ideal("x2 * x3", "x1 * x3", "x1 * x4", "x2**2", "x1 * x2", "x1**2", gen_out=True)
inv = inv_finder(ideal)
print("invariant =", inv)
test_admisibility(ideal, inv)
weights = make_weights(inv)
print("Weights^{-1}", weights)
blow_up(ideal, generators, weights, chart_index=1, verbose = True)
blow_up(ideal, generators, weights, chart_index=0, verbose = True)
ideal

invariant = [Fraction(2, 1), Fraction(2, 1), Fraction(2, 1), Fraction(2, 1)]
Weights^{-1} (1, 1, 1, 1)
==== chart  x2 ====
{x1: u*y0, x2: u, x3: u*y2, x4: u*y3}
[y2, y0*y2, y0*y3, 1, y0, y0**2]
[u**2*y2, u**2*y0*y2, u**2*y0*y3, u**2, u**2*y0, u**2*y0**2]
==== chart  x1 ====
{x1: u, x2: u*y1, x3: u*y2, x4: u*y3}
[y1*y2, y2, y3, y1**2, y1, 1]
[u**2*y1*y2, u**2*y2, u**2*y3, u**2*y1**2, u**2*y1, u**2]


<x2*x3,x1*x3,x1*x4,x2**2,x1*x2,x1**2>

In [99]:
import numpy as np
w0 = np.zeros(4)
w1 = np.array([1,0,0,0])
w2 = np.array([*sympy.symbols("b12,b22"), 0, 0])
w3 = np.array([*sympy.symbols("b13,b23,b33"), 0])
w4 = np.array([*sympy.symbols("b14,b24,b34,b44")])
tt = np.array([sympy.symbols("t")])
T = np.hstack((w1, np.outer(w0, w0).flatten() , np.outer(w0, np.outer(w0,w0)).flatten()))
U = np.hstack((w2, np.outer(tt*w1, w1).flatten() , np.outer(w0, np.outer(w0,w0)).flatten()))
V = np.hstack((w3, np.outer(tt*w1, w2).flatten() , np.outer(tt*tt*w1, np.outer(w1,w1)).flatten()))
W = np.hstack((w4, np.outer(tt*w1, w3).flatten() + np.outer(tt*w2, w2).flatten(), np.outer(tt*tt*w1, np.outer(w1,w2)).flatten()))
M = np.vstack((T,U,V,W))
MM = np.hstack((M,tt*tt*tt*np.array([0,0,0,1]).reshape(4,1)))
ring = [x for x in np.hstack((w0,w1,w2,w3,w4,tt)) if type(x)==sympy.Symbol]

In [100]:
from itertools import combinations
def minors2d(minor_size:int, matrix:np.ndarray, reduce = True, nonzero = True, asideal = True):
    if reduce: #reduce complexity by removing 0 cols
        matrix = matrix[:, ~np.all(matrix == 0, axis = 0)]
    nrows, ncols = matrix.shape
    assert nrows >= minor_size and ncols >= minor_size
    out = []
    row_choices = combinations(range(nrows), minor_size)
    col_choices = combinations(range(ncols), minor_size)
    for minor_row in row_choices:
        for minor_col in col_choices:
            minor = sympy.det(sympy.Matrix(matrix[minor_row,:][:,minor_col]))
            if minor != 0:
                out.append(minor)
    return out
def make_monomial(ideal:list):
    out = ",".join(str(s) for gen in ideal for s in sympy.Add.make_args(sympy.expand(gen)))
    return out

In [101]:
len(make_monomial(minors2d(4,MM)))

1239

In [ ]:
multiblowup(make_monomial(minors2d(4,MM)), verbose=1, path=[4,6,7,8], stop_depth=4)